In [289]:
import csv
training_data = []
with open('heart_train.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)
    for row in reader:
        training_data.append(row)

for row in training_data:
    print(row)
        



['29', '1', '1', '130', '204', '0', '0', '202', '0', '0', '2', '0', '2', '1']
['40', '1', '3', '140', '199', '0', '1', '178', '1', '1.4', '2', '0', '3', '1']
['41', '0', '1', '105', '198', '0', '1', '168', '0', '0', '2', '1', '2', '1']
['41', '1', '0', '110', '172', '0', '0', '158', '0', '0', '2', '0', '3', '0']
['41', '1', '1', '120', '157', '0', '1', '182', '0', '0', '2', '0', '2', '1']
['41', '1', '1', '135', '203', '0', '1', '132', '0', '0', '1', '0', '1', '1']
['42', '0', '0', '102', '265', '0', '0', '122', '0', '0.6', '1', '0', '2', '1']
['42', '1', '1', '120', '295', '0', '1', '162', '0', '0', '2', '0', '2', '1']
['43', '1', '0', '115', '303', '0', '1', '181', '0', '1.2', '1', '0', '2', '1']
['43', '1', '0', '150', '247', '0', '1', '171', '0', '1.5', '2', '0', '2', '1']
['44', '1', '0', '120', '169', '0', '1', '144', '1', '2.8', '0', '0', '1', '0']
['44', '1', '2', '130', '233', '0', '1', '179', '1', '0.4', '2', '0', '2', '1']
['45', '1', '0', '104', '208', '0', '0', '148', '1',

In [264]:
# For Python 2 / 3 compatability
from __future__ import print_function


In [265]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

In [266]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [267]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [268]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, question_column, question_value, question_header):
        self.column = question_column
        self.value = question_value
        self.q_header = question_header

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            self.q_header[self.column], condition, str(self.value))

In [269]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [270]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [271]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [272]:
def find_best_split(rows, new_header):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(new_header) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val, new_header)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [273]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [274]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [275]:
def build_tree(rows, new_header):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows,new_header)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows, new_header)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows, new_header)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [276]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [310]:
my_tree = build_tree(training_data, header)

IndexError: list index out of range

In [278]:
print_tree(my_tree)

Is thall == 3?
--> True:
  Is cp == 0?
  --> True:
    Predict {'0': 25}
  --> False:
    Is slp == 2?
    --> True:
      Predict {'1': 4}
    --> False:
      Is oldpeak == 1.9?
      --> True:
        Predict {'1': 1}
      --> False:
        Predict {'0': 7}
--> False:
  Is caa == 0?
  --> True:
    Is oldpeak == 2.8?
    --> True:
      Predict {'0': 1}
    --> False:
      Is thalachh == 154?
      --> True:
        Predict {'0': 1}
      --> False:
        Is chol == 335?
        --> True:
          Predict {'0': 1}
        --> False:
          Predict {'1': 40}
  --> False:
    Is cp == 0?
    --> True:
      Is oldpeak == 2?
      --> True:
        Predict {'1': 1}
      --> False:
        Predict {'0': 8}
    --> False:
      Is oldpeak == 0?
      --> True:
        Predict {'1': 7}
      --> False:
        Is slp == 1?
        --> True:
          Predict {'0': 2}
        --> False:
          Is caa == 3?
          --> True:
            Predict {'0': 1}
          --> False:
 

In [279]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [280]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [281]:
testing_data = []
with open('heart_test.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)

    for row in reader:
        testing_data.append(row)
print(len(testing_data))

202


In [286]:
good = 0
wrong = 0
for row in testing_data:
    if '0' in print_leaf(classify(row, my_tree)):
        value = 0
    else:
        value = 1
    if row[-1]==str(value):
        good = good + 1
    else:
        wrong = wrong +1

print(good)
print(wrong)

160
42


In [283]:
import random
def Reg_Tree(data, header, training_size, tree_arg):
  new_header = []
  for head in header:
    new_header.append(head)
  arg_len=len(data[1])
  new_training=[]
  for size_n in range(training_size):
      row = random.randint(0, len(data)-size_n)
      new_training.append(data[row])
  arg_to_delete = random.sample(range(arg_len-1),tree_arg)
  arg_to_delete.sort(reverse=True)
  for column in arg_to_delete:
      del new_header[column]
  for training_row in new_training:
      for column in arg_to_delete:
          del training_row[column]
  one_tree = build_tree(new_training, new_header)
  return one_tree
  
    

In [313]:
Tree = Reg_Tree(training_data, header, 50, 2)


['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh', 'exng', 'oldpeak', 'slp', 'caa', 'thall', 'output']


In [ ]:
print(len(header))
print(header)
print(training_data[1])
print(len(training_data[1]))

14
['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh', 'exng', 'oldpeak', 'slp', 'caa', 'thall', 'output']
['40', '1', '3', '140', '199', '0', '1', '178', '1', '1.4', '2', '0', '3', '1']
14


In [ ]:
def Reg_Forest(data, header, width,training_size,tree_arg):
  Forest = []
  for t in range(width):
    Forest[t]=Reg_Tree(data,header,training_size,tree_arg)
  return Forest
  



In [ ]:
My_Forest = Reg_Forest(training_data, header, 5, 80, 1)


IndexError: list index out of range